# PS 88 - Lab 9 - Multivariate Regression

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy import stats

## Part 1: Controlling with simulation

To see how controlling for confounding variables can help us estimate causal effects, we will first use some simulated data, and then move on to real data in part 2.

The political science motivation behind the simulations is the relationship between whether protest movements use violence and whether they succeed at attaining their goals. A <a href="http://cup.columbia.edu/book/why-civil-resistance-works/9780231156820">prominent finding</a> in this literature is that nonviolent movements tend to be more likely to achieve their goals. Of course, the choice to use violence is not random, so this may not be a causal effect.

One potential confounding variable is how repressive the government is. In particular, we may  be concerned that violence is more likely to be used when the government is more repressive, and repressive governments are less likely to grant concessions. 

Here is a simulation of this process. There  are three parameters which we will vary later:
- `b_rep` is effect of repression on whether the protest  movement succeeds. We assume this is negative, meaning success is harder with a more repressive government
- `b_viol` is the effect of violence of movement success. For the first simulation, we set this to zero, meaning there is no real causal effect
- `b_rv` is the effect of repressiveness on the use of violence, which we assume is positive

The following code simulates 1000 protest movements, with three variables we will observe: 
- `rep` is the repressiveness, which we will assume is normally distributed
- `viol` is equal to 1 for a violent movement and 0 for nonviolent. We assume movments are more likely to be violent with a repressive government.
- `succ` is a continuous measure of success, where higher numbers indicate a more successful movement. This is (potentially) a function of repressiveness, the choice of violent tactics, and random noise.

Let's first recreate the data we used in class:

In [ ]:
np.random.seed(89)
b_rep = -1
b_viol = 0
b_rv = 1
# Random repressiveness levels
rep = np.random.normal(0,1,1000)
viol = np.where(b_rv*rep + np.random.normal(0,1,1000) > 0, 1, 0)
succ = b_rep*rep + b_viol*viol + np.random.normal(0,.3,1000)
protest = pd.DataFrame(data={'Repressive': rep, 
                             'Violent': viol, 
                             'Success': succ})
protest

Recall that in these data we found that there is a negative relationship between violence and success even though there is no causal effect:

In [ ]:
succ_ols = smf.ols('Success~Violent', data=protest).fit()
succ_ols.summary()

Now let's consider a case where there is a causal effect of violence on protest. In particular, let's suppose violence leads to less success. To capture this, we can set `b_viol = -.2` and simulate data using a similar procedure. Note that in this case we explicitly compute the potential outcomes with and without violence, putting these in columns `Success0` and `Success1`.

In [ ]:
np.random.seed(99)
b_rep = -1
b_viol = -.2
b_rv = 1
rep = np.random.normal(0,1,1000)
viol = np.where(b_rv*rep + np.random.normal(0,1,1000) > 0,1,0)
succ0 = b_rep*rep + np.random.normal(0,.3,1000)
succ1 = succ0 + b_viol
succ = np.where(viol==1, succ1, succ0)
protest2 = pd.DataFrame(data={'Repressive': rep, 
                                 'Violent': viol, 
                                 'Success': succ,
                                'Success0':succ0,
                                'Success1':succ1})
protest2

**Question 1.1. Fit and summarize a bivariate regression where `Success` is the dependent variable and ``Violent`` is the independent variable on the `protest2` data. Interpret the coefficient on `Violent`, and compare it to what we got on the `protest` data from the class example.**

In [ ]:
#Code for 1.1

*Words for 1.1*

Recall that if we want to pull the values of `Var1` for the subset of rows where `Var2=x` from a data frame `df`, the code is:
`df.loc[df['Var2']==x, Var1]`. Think of `.loc` as a combination of what the `.where` and `.column` functions do in the Table library.

**Question 1.2. Compute the difference in means of the potential outcome of success if nonviolent (`Success0`) for movements that were violent versus not. How does this relate to the "difference of means = causal effect + selection bias" formula? (Recall the causal effect here is -.2, and the regression coefficient on `Violent` corresponds to a difference in means).**

In [ ]:
# Code for 1.2


*Words for 1.2*

**Question 1.3. Fit and summarize a multivaraite regression model where `Success` is the dependent variable and `Violent` and `Repressive` are independent variables. Interpret the coefficients on both of these variables. How does the coefficient on `Violent` compare to the bivariate model from question 1.1?**

In [ ]:
# Code for 1.3

*Words for 1.3*

Here is a function which does a simulation like the ones above, which takes the parameters that represent the real causal effects as arguments, and then returns the coefficient on `Violent` in the bivariate regression and the multivariate regression (controlling for `Repressive`). 

In [ ]:
def sim_protest(b_rep, b_viol, b_rv):
    rep = np.random.normal(0,1,1000)
    viol = np.where(b_rv*rep + np.random.normal(0,1,1000) > 0,1,0)
    succ0 = b_rep*rep + np.random.normal(0,.3,1000)
    succ1 = succ0 + b_viol
    succ = np.where(viol==1, succ1, succ0)
    protest = pd.DataFrame(data={'Repressive': rep, 
                                 'Violent': viol, 
                                 'Success': succ,
                                'Success0':succ0,
                                'Success1':succ1})
    mnocon = smf.ols('Success ~ Violent', data=protest).fit()
    mcon = smf.ols('Success ~ Violent + Repressive', data=protest).fit()
    return("Bivariate: " + str(mnocon.params[1]) + 
           ", Multivariate: "+ str(mcon.params[1]))

Here is a simulation with our initial parameters, with no real causal effect of violence. This should give similar results (not exactly the same since there is some randomness in the data generation):

In [ ]:
sim_protest(b_rep=-1,b_viol=0,b_rv=1)

**Question 1.4 Use the `sim_protest` function to show that even if violence makes movements more successful, we could still observe a negative coefficient in the bivariate regression with `Success` as the dependent variable and `Violent` as the independent variable. (Hint: set `b_viol` to something positive but not too large) Compare this with the coefficient you get from the multivariate regression.**

In [ ]:
# Code for 1.4

* Words for 1.4*

**Question 1.5. Now run a simulations where `b_rv = 0`, meaning repressiveness does not affect the violence choice. How do the coefficients between in the bivariate and multivariate regression compare now, and why? (Hint: think about the conditions for a confounding variable to cause selection bias.)**

In [ ]:
# Code for 1.5

*Words for 1.5*

## Part 2: Voting for the ACA and 2010 Vote Share as Difference of Means/Regression
In this part we are going to look at political implications of the <a href='https://en.wikipedia.org/wiki/Affordable_Care_Act'>Affordable Care Act</a> which was passed early in Obama's first term. This was one of the more consequential pieces of domestic legislation in decades, and the Democrats lost 63 seats in the House of Representatives in the 2010 midterms after the passing of the ACA. Let's explore if there is evidence that this was driven by votes on the bill.

First, we are going to load up some data on the midterm elections in the House in 2010. The data come from <a href="https://journals.sagepub.com/doi/abs/10.1177/1532673X11433768">this paper</a>, and are stored in Stata format. We can read this in using the `read_stata` function from the pandas library.

In [ ]:
hcr_mid = pd.read_stata("hcr_midterm.dta")
hcr_mid

How someone voted on the ACA is stored in the variable `hcr_yes`, and whether they are a Republican or Democrat is in `party`. We can look at the relationship between these two with the `crosstab` function in pandas, which is like a pivot table.

In [ ]:
pd.crosstab(hcr_mid["hcr_yes"], hcr_mid["party"])

This tells us that only <a href="https://en.wikipedia.org/wiki/Joseph_Cao">one Republican</a> voted for the bill, and 39 democrats voted against it. The main comparison we want to make is whether the Democrats who voted for the bill did better or worse in the 2010 midterms than those who voted against it.

To do this, let's first subset the data to districts with democratic incumbents who ran in competitive elections (i.e., with an opponent). To do this, we get a subset of the data where two conditions are met: `hcr_mid["dem_n"] > 0` means the election was competitive (uncontested seats with only one candidate get coded as a zero in the data file) and `hcr_mid["party"]=="D")` ensures they are democrats. We use the `&` to ensure both are true.

In [ ]:
hcr_mid = hcr_mid[(hcr_mid["dem_n"] > 0) & (hcr_mid["party"]=="D")]
hcr_mid

The `dem_n` variable is the democratic candidate vote share in the 2010 midterms. Let's look at the distribution:

In [ ]:
np.mean(hcr_mid["dem_n"])

In [ ]:
sns.displot(hcr_mid["dem_n"])
plt.axvline(50)

The average D running for re-election recieved almost 60% of the vote, but quite a few lost re-election.

Now let's compare the performance of those who voted for and against the bill. First, let's create separate data frames for the Y and N voters. (Note: we could also keep them together and use the `.loc` function like above when computing means, but it's always good to see multiple ways to do the same thing!)

In [ ]:
hcr_mid_yea = hcr_mid[hcr_mid["hcr_yes"] == 1]
hcr_mid_nay = hcr_mid[hcr_mid["hcr_yes"] == 0]

And take the average vote shares among these two groups.

In [ ]:
mean_yea = np.mean(hcr_mid_yea["dem_n"])
mean_yea

In [ ]:
mean_nay = np.mean(hcr_mid_nay["dem_n"])
mean_nay

Here is the raw difference of means:

In [ ]:
dom = mean_yea - mean_nay
dom

**Question 2.1 Use `smf.ols` to run a bivariate regression with `dem_n` as the dependent variable and `hcr_yes` as the independent variable. Save the fitted model as `ols_model`, and then use `.summary()` to get the output. (Hint: see the code between questions 1.1 and 1.2 for an example of what this should look like.) Compare this with the means/difference of means above.**

In [ ]:
# Code for 2.1

*Words for 2.1*

Now let's think about some reasons why this relationship might not be causal. A major confounding variable is that those who voted for the bill likely represent more liberal districts, making their re-election easier. To check this, we will look also bring Obama's 2008 vote share into our analysis.

First, let's look at the relationship between Obama's 2008 vote share and the House members 2010 vote share.

In [ ]:
sns.scatterplot(x='obama', y='dem_n', data=hcr_mid)

As we can see, there is a strong positive relationship. This shouldn't surprise us: most people vote for the same party consistently.



Let's look at the relationship between Obama vote share and voting for the ACA.

In [ ]:
sns.scatterplot(x='obama', y='hcr_yes', data=hcr_mid)

As with some of our simulated data, this looks a little goofy because the hcr_yes variable just takes on value of 0 or 1. We can still run a "linear probability model" with Obama vote share as the independent ($X$) variable and the vote as the dependent ($Y$) variable.

**Question 2.2. Use `smf.ols` to run a linear probability model with Obama vote share as the independent variable and whether the member of congress voted for the ACA as the dependent variable, save the fitted model as `vote_model`, then use the `.summary()` function to see the output. Interpret the slope on the `obama` variable.**

In [ ]:
#Code for 2.2

Since the DV here is binary, we can interpret the slope as meaning "as Obama vote share goes up by 1%, the probability of voting for the ACA increases by 1.4%.

We can plot this prediction (note: this assumes you saved the `smf.ols` output as `vote_model`!


In [ ]:
sns.scatterplot(x='obama', y='hcr_yes', data=hcr_mid)
xrange = np.arange(30, 100)
plt.plot(xrange, vote_model.params[0] + xrange*vote_model.params[1])

Now visualize the three variable together, but doing a scatter plot of Obama vote share and 2010 Democratic vote share, with green dots for those who voted Y and orange dots for those who voted N.

In [ ]:
sns.scatterplot(x='obama', y='dem_n', data=hcr_mid_y, color="green")
sns.scatterplot(x='obama', y='dem_n', data=hcr_mid_n, color="orange")

Note the orange dots are to the left, meaning those who voted N where generally in more moderate/conservative districts.

**Question 2.3. Receate the scatterplot above, and add horizontal lines corresponing to the mean vote share of those who voted No and the mean vote share of those who voted Yes, using the `plt.axhline` function (see the last plot above question 1.2 for an example of this).**

In [ ]:
# Code for 2.3

And we can look at the relationship between the vote and how liberal the district was by plotting the average Obama vote share among the Y and N districts:

In [ ]:
sns.scatterplot(x='obama', y='dem_n', data=hcr_mid_y, color="green")
sns.scatterplot(x='obama', y='dem_n', data=hcr_mid_n, color="orange")
plt.axvline(np.mean(hcr_mid_y["obama"]), color='green')
plt.axvline(np.mean(hcr_mid_n["obama"]), color='orange')

Combining what we know so far: districts where members voted fr the ACA did much better in the election, but these were also just more liberal ("safe") disticts. 

Now lets use mulitvariate regression to "control" for Obama vote share.

First, let's re-do our bivariate analysis using `smf.ols`.

In [ ]:
ols_biv = smf.ols('dem_n ~ hcr_yes', data=hcr_mid).fit()
ols_biv.summary()

**Question 2.4. Now fit and summaraze a multivariate regression with both `hcr_yes` and `obama` as independent variables. Save the fitted model as `ols_mv`, and then use `.summary()` to get the output. How does controlling for Obama vote share change the coefficient on the `hcr_yes` variable?**

In [ ]:
# Code for 2.4

*Words for 2.4*

To make a plot, we can pull out the coefficients by adding a `.params` after the name of our fitted model (note: this assumes you named the fitted model `ols_mv`).

In [ ]:
ols_mv.params

Let's overlay the predicted value for Y and N votes as a function of Obama vote share.

In [ ]:
sns.scatterplot(x='obama', y='dem_n', data=hcr_mid_y, color="green")
sns.scatterplot(x='obama', y='dem_n', data=hcr_mid_n, color="orange")
xrange = np.arange(30, 100)
plt.plot(xrange, ols_mv.params[0] + xrange*ols_mv.params[2], color="orange")
plt.plot(xrange, ols_mv.params[0] + ols_mv.params[1] + xrange*ols_mv.params[2], color="green")

This is nice illustration of what we mean by "controlling for Obama vote share" or "holding Obama vote share constant". The model accounts for both of these variables, and so the prediction is that *for a fixed level of Obama vote share*, those who vote N do better. Visually, there are two parallel lines, and the N line is always higher. 

Knowing the Obama vote share for each Democrat in the house, the model does not predict that those who vote N will do better in general, because they tend to be in more conservative districts. But it does predict for any two members in districts where Obama got the same vote share, but one votes Y and one votes N, the one voting N will do better.

The takeaway is that the main predictor of how well D's running did in 2010 is how well Obama did in 2008. The fact that those who voted for the ACA did better overall is misleading!

This model predicts that in a counterfactual world where those who voted against the ACA voted for it, they would have done worse in their re-election bids. Conversely, those who voted for the ACA could have done better (and some likely would have been re-elected) if they voted against it. 

Some important caveats:
- This may not be the "right" model: there might be remaining confounding variables.
- We are implicitly assuming that the effect of voting for the ACA is the same for everyone, which is probably not true: in very liberal disticts voters probably would have been unappy in their representative voted no!